In [14]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF
from pandas.io.json import json_normalize
from pymongo import MongoClient


In [3]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.MovieRecommender
ratings = db.ratings
movie_info = db.movies_metadata

In [9]:
data = list(ratings.aggregate([{'$sample': 50}]))

OperationFailure: the $sample stage specification must be an object

In [7]:
ratings_df = json_normalize(data)

,_id,_omdb_id,rating,timestamp,userId
0,5c83f32cfc0c4b2eee12bea2,110,1.0,1425941529,1
1,5c83f32cfc0c4b2eee12bea3,147,4.5,1425942435,1
2,5c83f32cfc0c4b2eee12bea4,858,5.0,1425941523,1
3,5c83f32cfc0c4b2eee12bea5,1221,5.0,1425941546,1
4,5c83f32cfc0c4b2eee12bea6,1246,5.0,1425941556,1
5,5c83f32cfc0c4b2eee12bea7,1968,4.0,1425942148,1
6,5c83f32cfc0c4b2eee12bea8,2762,4.5,1425941300,1
7,5c83f32cfc0c4b2eee12bea9,2918,5.0,1425941593,1
8,5c83f32cfc0c4b2eee12beaa,2959,4.0,1425941601,1
9,5c83f32cfc0c4b2eee12beab,4226,4.0,1425942228,1


# CSVs

In [38]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')#.fillna(-1).astype('int64')

In [234]:
ratings_df = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')
movies_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])

###May need Fuzz matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]


ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')




/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#All Ratings matched with movies
movies_df.merge(ratings_df, how='right', left_on='id', right_on='movieId')

#All Movies regardless if they were rated

In [69]:
movies_df.merge(ratings_df, how='left', left_on='id', right_on='movieId')

,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,poster_path,...,status,tagline,title,video,vote_average,vote_count,userId,movieId,rating,timestamp
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,Released,NaN,Toy Story,False,7.7,5415.0,NaN,NaN,NaN,NaN
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,NaN,NaN,NaN,NaN
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,NaN,NaN,NaN,NaN
5,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,23.0,949.0,3.5,1.148721e+09
6,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,102.0,949.0,4.0,9.565989e+08
7,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,232.0,949.0,2.0,9.550927e+08
8,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,242.0,949.0,5.0,9.566888e+08
9,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,263.0,949.0,3.0,1.117847e+09


In [65]:
movies_df.merge(ratings_df, how='right', left_on='id', right_on='movieId').shape

(100009, 25)

In [72]:
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')

In [73]:
ratings_with_movie_names.head()

,userId,movieId,rating,timestamp,id,original_title
0,1,31,2.5,1260759144,NaN,NaN
1,1,1029,3.0,1260759179,NaN,NaN
2,1,1061,3.0,1260759182,NaN,NaN
3,1,1129,2.0,1260759185,NaN,NaN
4,1,1172,4.0,1260759205,NaN,NaN


In [83]:
imdb_ratings_with_movie_names = ratings_df.merge(movies_df[['imdb_id', 'original_title']], how='left', left_on='movieId', right_on='imdb_id')

In [84]:
imdb_ratings_with_movie_names.shape

(100004, 6)

In [85]:
imdb_ratings_with_movie_names.head()

,userId,movieId,rating,timestamp,imdb_id,original_title
0,1,31,2.5,1260759144,NaN,NaN
1,1,1029,3.0,1260759179,NaN,NaN
2,1,1061,3.0,1260759182,NaN,NaN
3,1,1129,2.0,1260759185,NaN,NaN
4,1,1172,4.0,1260759205,NaN,NaN


In [86]:
imdb_ratings_with_movie_names[imdb_ratings_with_movie_names.original_title.isnull() == True].shape

(100004, 6)

In [90]:
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
legit_ratings = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

In [77]:
ratings_with_movie_names.head()

,userId,movieId,rating,timestamp,id,original_title
0,1,31,2.5,1260759144,NaN,NaN
1,1,1029,3.0,1260759179,NaN,NaN
2,1,1061,3.0,1260759182,NaN,NaN
3,1,1129,2.0,1260759185,NaN,NaN
4,1,1172,4.0,1260759205,NaN,NaN


In [118]:
legit_ratings.tail()

,userId,movieId,rating,timestamp,id,original_title
99958,671,2959,4.0,1064890427,2959.0,License to Wed
99960,671,3052,1.0,1063503966,3052.0,Don't Make Waves
99974,671,4011,4.0,1064245574,4011.0,Beetlejuice
99983,671,4886,5.0,1064245488,4886.0,And Then There Were None
99988,671,4995,4.0,1064891537,4995.0,Boogie Nights


In [229]:
len(legit_ratings.original_title.unique())
len(ratings_with_movie_names.original_title.unique())

1389

In [236]:
# We'll use the famous SVD algorithm.
algo = SVD(verbose=True)
algo2 = KNNWithZScore()
algo3 = KNNBaseline(bsl_options={'method': 'sgd'})
algo4 = NMF()


reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [244]:
from surprise.prediction_algorithms.knns import KNNBasic
sim_options = {'name': 'cosine',
               'user_based': True  # compute  similarities between items
               }
algo = SVD(verbose=True)

In [245]:
algo.fit(trainset)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8938  0.8942  0.9075  0.8940  0.8927  0.8964  0.0056  
MAE (testset)     0.6874  0.6909  0.6952  0.6893  0.6868  0.6899  0.0030  
Fit time          4.49    4.29    3.99    3.82    3.66    4.05    0.30    
Test time         0.13    0.12    0.13    0.13    0.12    0.13    0.00    


{'test_rmse': array([0.89375892, 0.89416475, 0.907531  , 0.89395464, 0.89266232]),
 'test_mae': array([0.68741494, 0.69092396, 0.69524899, 0.68928817, 0.6868084 ]),
 'fit_time': (4.48611307144165,
  4.294166088104248,
  3.989053964614868,
  3.8179097175598145,
  3.663433074951172),
 'test_time': (0.1303701400756836,
  0.12392807006835938,
  0.1300520896911621,
  0.12836813926696777,
  0.12376284599304199)}

In [246]:
algo4.predict(1, 2959)

AttributeError: 'NMF' object has no attribute 'trainset'

In [137]:
from collections import defaultdict

In [182]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [247]:
testset = trainset.build_anti_testset()

In [248]:
predictions = algo.test(testset)

In [249]:
top_n = get_top_n(predictions, n=50)

#for uid, user_ratings in top_n.items():
#    print(uid, [iid for (iid, _) in user_ratings])

In [258]:
#top_n[20]
lower_suggested_films = top_n[1][-6:]

In [262]:
ratings_with_movie_names.groupby('userId').get_group(1)

,userId,movieId,rating,timestamp,id,original_title
0,1,31,2.5,1260759144,NaN,NaN
1,1,1029,3.0,1260759179,NaN,NaN
2,1,1061,3.0,1260759182,NaN,NaN
3,1,1129,2.0,1260759185,NaN,NaN
4,1,1172,4.0,1260759205,NaN,NaN
5,1,1263,2.0,1260759151,NaN,NaN
6,1,1287,2.0,1260759187,NaN,NaN
7,1,1293,2.0,1260759148,NaN,NaN
8,1,1339,3.5,1260759125,NaN,NaN
9,1,1343,2.0,1260759131,NaN,NaN


In [263]:
for pred in top_n[1]:
    movie_name = ratings_with_movie_names[ratings_with_movie_names.movieId == pred[0]]['original_title'].iloc[0]
    print(f'The Movie name is {movie_name}')

The Movie name is nan
The Movie name is The Good Thief
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is While You Were Sleeping
The Movie name is nan
The Movie name is Point Break
The Movie name is Galaxy Quest
The Movie name is nan
The Movie name is Nell
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is Murder She Said
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is The Million Dollar Hotel
The Movie name is Lonely Hearts
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is The Thomas Crown Affair
The Movie name is nan
The Movie name is nan
The Movie name is The Thomas Crown Affair
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is Totally Blonde
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is nan
The Movie name is Gremlins 2: The New Batch

In [257]:
top_n.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [ ]:
top_n[1]

In [264]:
ratings_with_movie_names[ratings_with_movie_names.movieId == 6016]['original_title'].iloc[0]

'The Good Thief'